## Building models with **paintbox**

In **paintbox**, the models used to describe the observed spectrum and/or spectral energy distribution of a galaxy are build from a combination of spectral components, including the stellar continuum, emission lines for the gas, etc. Moreover, the parametrization of the model, i.e., the specific details about how these spectral elements are combined, are defined interactively. Below, we illustrate how to generate these spectral components in practice and how to combine them to make a model.

### Using CvD models

Here we use the CvD single stellar popupation models, and the response functions for several elements, prepared in this [tutorial](https://paintbox.readthedocs.io/en/latest/preparing_models.html#preparing-cvd-models) to illustrate how to  create the basic elements for the stellar continuum. 

In [4]:
import os

from astropy.io import fits
from astropy.table import Table

import paintbox as pb

In [6]:
ssp_file = "templates/VCJ17_varydoublex.fits"
templates = fits.getdata(ssp_file, ext=0)

FileNotFoundError: [Errno 2] No such file or directory: 'templates/VCJ17_varydoublex.fits'